In [2]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
pages = np.arange(1, 9951, 50)

titles = []
years = []
ratings = []
genres = []
runtimes = []
imdb_ratings = []
values=[]
votes = []
gross = []
metascores=[]


for page in pages:
    
    #get request
    response = get("https://www.imdb.com/search/title/?sort=num_votes,desc&" 
                   + "start=" 
                   + str(page) 
                   + "&title_type=feature&year=2000,2020")
    page_html = BeautifulSoup(response.text, 'html.parser')
    page_concat = page_html.find_all('div', class_= 'lister-item mode-advanced')
    #print(type(page_concat)) #to double check that I got a ResultSet
    #print(len(page_concat)) #to double check I got 50 (elements/page)
    #initialize empty lists to store the variables scraped
    

    #extract data itemwise
    for container in page_concat:
    
    #conditional for all with metascore
        if container.find('div', class_ = 'ratings-metascore') is not None:
        
        #title
            title = container.h3.a.text
            titles.append(title)
        
     #year released
            year = container.h3.find('span', class_= 'lister-item-year text-muted unbold').text
            years.append(year)
        
            #rating
            if container.p.find('span', class_ = 'certificate') is not None: 
                rating = container.p.find('span', class_= 'certificate').text
                ratings.append(rating)
            else:
                ratings.append('NaN')
     

            #genre
            genre = container.p.find('span', class_ = 'genre').text
            genres.append(genre)

            #runtime
            if container.p.find('span', class_ = 'runtime') is not None: 
                time = container.p.find('span', class_ = 'runtime').text
                runtimes.append(time)
            else:
                runtimes.append('NaN')

            #IMDB ratings
            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)

            #Metascore
            m_score = container.find('span', class_ = 'metascore').text
            metascores.append(int(m_score))


            #Number of votes
            container_values = container.find_all('span', attrs = {'name':'nv'})
            del values[:]

            if len(container_values)==2:
                for value in range(2) :
                    values.append(container_values[value]['data-value'])
                votes.append(int(values[0]))
                gross.append(values[1])
            elif len(container_values)==1:
                votes.append(int(container_values[0]['data-value'][0]))
                gross.append('NaN')
            else :
                votes.append('NaN')
                gross.append('NaN')






In [46]:
df = pd.DataFrame({'Film': titles,
                       'Année': years,
                       'Catégorie': ratings,
                       'Genre': genres,
                       'Durée (min)': runtimes,
                       'Metascore': metascores,
                       'Score imdb': imdb_ratings,
                       'Nb votes': votes,
                       'CA(gross)': gross})


In [47]:
#remove the parenthesis at the front and the end of the year and turn them into integers
df.loc[:, 'Année'] = df['Année'].str[-5:-1].astype(int)

#standardize the IMDB score so that it is comparable to the scale of the other one
df['Score imdb'] = df['Score imdb'] * 10

#remove " min" from the end of the runtime variable and make them integers
df['Durée (min)'] = df['Durée (min)'].apply(lambda x: x.replace(" min", "")).apply(lambda x: int(x))

#drop the old runtime column
#df.drop(columns='Durée', axis=1, inplace=True)

#remove the "\n" at the beginning of the genres
df['Genre'] = df['Genre'].apply(lambda x: x.replace("\n", ""))

df['Genre'] = df['Genre'].apply(lambda x: x.replace("[", ""))

df['Genre'] = df['Genre'].apply(lambda x: x.replace("]", ""))

df['Genre'] = df['Genre'].apply(lambda x: x.replace(",", ""))


#after standardizing the imdb variable, it became a float. We don't need it to be so I made it an int
#df['n_imdb'] = df['n_imdb'].apply(lambda x: int(x))

#I found that there was whitespace in the arrays of genres so I removed that with .rstrip()
df['Genre'] = df['Genre'].apply(lambda x: x.rstrip())

#split the string list into an actual array of values to do NLP on later
df['Genre'] = df['Genre'].str.split()


df['CA(gross)']=df['CA(gross)'].str.replace(',', '')


In [48]:
df

,Film,Année,Catégorie,Genre,Durée (min),Metascore,Score imdb,Nb votes,CA(gross)
0,The Dark Knight: Le chevalier noir,2008,Tous publics,"[Action, Crime, Drama]",152,84,90.0,2266354,534858444
1,Inception,2010,Tous publics,"[Action, Adventure, Sci-Fi]",148,74,88.0,2028429,292576195
2,Le Seigneur des anneaux : La Communauté de l'a...,2001,Tous publics,"[Action, Adventure, Drama]",178,92,88.0,1633976,315544750
3,Le Seigneur des anneaux : Le Retour du roi,2003,Tous publics,"[Action, Adventure, Drama]",201,94,89.0,1618154,377845905
4,The Dark Knight Rises,2012,Tous publics,"[Action, Adventure]",164,78,84.0,1494684,448139099
...,...,...,...,...,...,...,...,...,...
6744,Trois souvenirs de ma jeunesse,2015,Tous publics,"[Drama, Romance]",123,87,68.0,3168,268548
6745,Amours cannibales,2013,Tous publics avec avertissement,"[Drama, Romance, Thriller]",116,69,58.0,3,NaN
6746,96 Minutes,2011,R,"[Drama, Thriller]",93,51,57.0,3,NaN
6747,Le chat du rabbin,2011,Tous publics,"[Animation, Comedy, Fantasy]",100,74,72.0,3166,16659


In [49]:
df.to_csv(r'C:\Users\Utilisateur\Documents\ENSAE COURS\ENSAE 2A\Projet Python 2A\DFscrap.csv',index = False)

In [50]:
df = pd.read_csv(r'C:\Users\Utilisateur\Documents\ENSAE COURS\ENSAE 2A\Projet Python 2A\DFscrap.csv')
